In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from requests_html import HTMLSession, AsyncHTMLSession

In [3]:
rejected = [
    'wa.link',
    'mailto:',
    't.me',
    'tally.so',
    'discord.gg',
    'arxiv.org',
    'javascript:'
]

In [4]:
async def get_html(url):
    session = AsyncHTMLSession(verify = False)
    result = None
    try:
        
        r = await session.get(url, verify = False, timeout = 10)
        await r.html.arender(timeout = 10)
        
        result = r.html.html
    except Exception as e:
        pass
    
    await session.close()
    return result


def get_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = []
    for link in soup.find_all('a'):
        links.append(link.get('href'))
    return links

In [5]:
import asyncio

visited_pages = set(['https://spa.gov.my/spa/'])

async def crawl(base_url):
    if base_url.endswith('.pdf'):
        return
    
    html = await get_html(base_url)
    
    if html is None:
        return

    visited_pages.add(base_url)
    links = get_links(html)
    for link in links:
        try:
            if any([r in link for r in rejected]):
                continue

            if not link.startswith('http'):
                link = urljoin(base_url, link)

            link = link.split('#')[0]
            link = link.split('?')[0]

            if link not in visited_pages:
                visited_pages.add(link)
        except Exception as e:
            pass

In [ ]:
from tqdm import tqdm

batch_size = 10

for no in range(2):
    l = list(visited_pages)
    print(no, len(l))
    for i in tqdm(range(0, len(l), batch_size)):
        b = l[i: i + batch_size]
        await asyncio.gather(*[crawl(url) for url in b])

0 3651


  0%|                                                   | 0/366 [00:00<?, ?it/s]/tmp/ipykernel_3640715/3492036454.py:11: RuntimeWarning: coroutine 'Launcher.killChrome' was never awaited
  pass
connection unexpectedly closed
Task exception was never retrieved
future: <Task finished name='Task-44304' coro=<Connection._async_send() done, defined at /home/husein/.local/lib/python3.8/site-packages/pyppeteer/connection.py:69> exception=InvalidStateError('invalid state')>
Traceback (most recent call last):
  File "/home/husein/.local/lib/python3.8/site-packages/websockets/legacy/protocol.py", line 968, in transfer_data
    message = await self.read_message()
  File "/home/husein/.local/lib/python3.8/site-packages/websockets/legacy/protocol.py", line 1038, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
  File "/home/husein/.local/lib/python3.8/site-packages/websockets/legacy/protocol.py", line 1113, in read_data_frame
    frame = await self.read_frame(max_size)

RuntimeError: This event loop is already running

In [7]:
import json

with open('url-spa.gov.my.json', 'w') as fopen:
    json.dump(list(visited_pages), fopen)